# EEG Viewer
![status](https://img.shields.io/badge/status-in%20progress-orange)



<div style="text-align: center;">
    <img src="https://raw.githubusercontent.com/holoviz-topics/neuro/main/workflows/eeg-viewer/assets/230524_eeg-viewer.png" alt="eeg viewer preview" width="450"/>
</div>

## Summary

This workflow is intended to demonstrate the visualization of a set of 1D EEG timeseries with HoloViz and Bokeh tools.

For details specific to this workflow, such as goals, specifications, and bottlenecks, please see this workflow's [readme](./readme_eeg-viewer.md).

For a summary of EEG research, data, and software, please see [neuro/wiki/EEG-notes](https://github.com/holoviz-topics/neuro/wiki/EEG-notes).

## Imports and config

<div class="admonition alert alert-info">
    <p class="admonition-title" style="font-weight:bold">Requirements</p>
    <p>This workflow notebook requires the <a href="./environment.yml">environment</a> specified in this workflow directory.</p>
</div>


In [ ]:
%load_ext autoreload
%autoreload 2

import colorcet as cc
import holoviews as hv
import mne
import numpy as np
import pandas as pd
import panel as pn
from bokeh.models import HoverTool, WheelZoomTool
from holoviews import Dataset
from holoviews.operation.datashader import rasterize
from holoviews.plotting.links import RangeToolLink
from hvneuro import download_file
from neurodatagen.eeg import generate_eeg_powerlaw
from scipy.stats import zscore

hv.extension("bokeh")
pn.extension(template="material")

## Real data pipeline

### Intake data
Downloading the data if it does not already exists. The data size is 2.6 MB. 

In [ ]:
url = "https://physionet.org/files/eegmmidb/1.0.0/S001/S001R04.edf?download"
local_data_path = "../../data/"
local_file_path = download_file(url, local_data_path)

Load the data and show the info

In [ ]:
raw = mne.io.read_raw_edf(local_file_path, preload=True)
raw.info

Preview the channel names, types, signal ranges, and uncompressed size

In [ ]:
raw.describe()

### Clean channel names, set sensor positions, and reference data

In [ ]:
# clean up the channel names
raw.rename_channels(lambda s: s.strip("."));

In [ ]:
# # preview available montages that are shipped with MNE
# mne.channels.get_builtin_montages(descriptions=True)

In [ ]:
# # Let's use the standard 10-20
# montage = mne.channels.make_standard_montage("standard_1020")

In [ ]:
# # plot the assigned positions of our data channels
# raw.set_montage(montage, match_case=False)
# sphere=(0, 0.015, 0, 0.099) #manually adjust the y origin coord and radius a bit
# raw.plot_sensors(show_names=True, sphere=sphere);

In [ ]:
# re-reference EEG data to the average over all recording channels
raw.set_eeg_reference("average");

### Gather the data for plotting into simple arrays

In [ ]:
time = raw.times
channels = raw.ch_names

# get the EEG data (for this data set, all channels are EEG anyways)
eeg_indices = mne.pick_types(raw.info, eeg=True)
data = raw.get_data(picks=eeg_indices, units={"eeg": "uV"})

### Gather the annotations
Getting the annotation data and clean it up.

In [ ]:
# Get annotations into a Pandas DataFrame
annotations = raw.annotations.to_data_frame()
annotations["start_time"] = annotations["onset"]
duration_seconds = pd.to_timedelta(annotations["duration"], "s")
annotations["end_time"] = annotations["start_time"] + duration_seconds
annotations = annotations.drop(["onset", "duration"], axis=1)

# For now we convert it too seconds since orig_time
orig_time = raw.annotations.orig_time.replace(tzinfo=None)
annotations["start_time"] = (annotations["start_time"] - orig_time).dt.total_seconds()
annotations["end_time"] = (annotations["end_time"] - orig_time).dt.total_seconds()

unique_descriptions = annotations["description"].unique()
color_map = dict(zip(unique_descriptions, cc.glasbey))

annotations.head()

In [ ]:
from holonote.annotate import Annotator

# Want to add more automatic way to set this up.
# annotator = Annotator(spec={"time": np.datetime64}, fields=["description"])
annotator = Annotator(spec={"time": float}, fields=["description"])

if annotator.df.empty:
    # Only run once!
    for n in annotations.itertuples():
        annotator.set_range(n.start_time, n.end_time)
        annotator.add_annotation(description=n.description)
    annotator.commit()


# This does not work with annotations yet
range_style = {
    "color": hv.dim("description").categorize(color_map),
    "show_legend": False,
}
range_style = {"show_legend": False}

# Using an empty Curve to makes it possible to reuse the overlay.
annotations_overlay = annotator.overlay(hv.Curve([]), range_style=range_style)

### Visualize real data
Plotting constants and tools for the next plots

In [ ]:
max_ch_disp = 10  # max channels to initially display
max_t_disp = 5  # max time in seconds to initially display

spacing = 2.5  # Spacing between channels
offset = np.std(data) * spacing

y_positions = np.arange(len(channels)) * offset
yticks = list(zip(y_positions, channels))

clim_spacing = 1.2

hover = HoverTool(
    tooltips=[
        ("Channel", "@channel"),
        ("Time", "$x s"),
        ("Amplitude", "@original_amplitude µV"),
    ]
)
wheel = WheelZoomTool(
    zoom_together="none",
    dimensions="width",
    maintain_focus=False,
)
tools = ["save", "pan", wheel, "box_zoom", "reset", hover]

Create the EEG viewer and applying annotation to it

In [ ]:
# Create eeg_viewer
data_with_offset = data + (np.arange(len(data))[:, np.newaxis] * offset)
max_data = data_with_offset.max()
ds = hv.Dataset(
    (channels, time, data_with_offset.T, data.T),
    kdims=["channel", "Time"],
    # vdims=["Amplitude", "original_amplitude"],  # Original amplitude does not work great with HoverTool in an overlay plot.
    vdims=["Amplitude"],
)
eeg_viewer = (
    ds.to(hv.Curve, groupby="channel")
    .overlay()
    .opts(hv.opts.Curve(color="black", line_width=1, tools=[hover, "xwheel_zoom"]))
)

# Combine with annotations
eeg_with_annotations = annotations_overlay * eeg_viewer

# Style the EEG Viewer
eeg_with_annotations = eeg_with_annotations.opts(
    padding=0,
    xlabel="Time (s)",
    ylabel="Channel",
    yticks=yticks,
    show_legend=False,
    aspect=1.5,
    responsive=True,
    shared_axes=False,
    backend_opts={
        "x_range.bounds": (time.min(), time.max()),
        "y_range.bounds": (data.min(), max_data),
    },
)

Create the minimap, this is done by [rasterizing](https://holoviews.org/user_guide/Large_Data.html#holoviews-operations-for-datashading) it.

In [ ]:
# Compute z-scores across time for each channel
z_data = zscore(data, axis=1)

# Generate the zscored image for the minimap using the y tiack positions from the eeg_viewer
minimap = rasterize(
    hv.Image((time, y_positions, z_data), ["Time (s)", "Channel"], "Amplitude (uV)")
)

# Style the minimap
minimap = minimap.opts(
    cmap="RdBu_r",
    colorbar=False,
    xlabel="",
    alpha=0.5,
    yticks=[yticks[0], yticks[-1]],
    height=100,
    responsive=True,
    default_tools=[""],
    shared_axes=False,
    clim=(-z_data.std() * clim_spacing, z_data.std() * clim_spacing),
    xlim=(time.min(), time.max()),  # If annotations exceed the time of the plot
)

Create RangeToolLink between the minimap and the main EEG viewer

In [ ]:
max_y_disp = data_with_offset[max_ch_disp - 1].max()
link = RangeToolLink(
    minimap,
    eeg_viewer,
    axes=["x", "y"],
    boundsx=(None, max_t_disp),
    boundsy=(None, max_y_disp),
)

Combine the EEG Viewer with the minimap and making it a Panel app 

In [ ]:
eeg_with_minimap = (eeg_with_annotations + minimap * annotations_overlay).cols(1)

eeg_app = pn.panel(eeg_with_minimap, min_height=650).servable(
    target="main", title="EEG Viewer with HoloViz and Bokeh"
)
eeg_app

Select a range in the plot by clicking around, and add an annotation with the cell below 

In [ ]:
annotator.add_annotation(description="Hello")

Saving it to the database can be done with this:

In [ ]:
annotator.commit()